In [1]:
import numpy as np, pandas as pd, tensorflow as tf
from sentence_transformers import SentenceTransformer
from tensorflow.keras import layers, losses, Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
import scipy.stats as stats
from math import ceil
from sklearn.preprocessing import MinMaxScaler

In [2]:
#read the text file
file = open("yelp/train.txt", "r", encoding = "utf8")
file_data = file.read()
lines = file_data.splitlines()
file.close()

data = []

#only use senteces that contain the keyword
keyword = ""
for s in lines:
    if keyword in s:
        data.append(s)

#appending a random weird sentence to see if it pops to the top of the sentence rankings        
#data.append("Colorless green ideas sleep furiously .")

#data = formatting(data)

#generates embedding for the sentences
emb = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')
#emb = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embed = emb.encode(data, show_progress_bar = True)

Batches:   0%|          | 0/6250 [00:00<?, ?it/s]

In [212]:
file = open("yelp/train.txt", "r", encoding = "utf8")
file_data = file.read()
lines = file_data.splitlines()
file.close()

data = []

#only use senteces that contain the keyword
keyword = "pizza"
for s in lines:
    if keyword in s:
        data.append(s)

#appending a random weird sentence to see if it pops to the top of the sentence rankings        
#data.append("Colorless green ideas sleep furiously .")

#data = formatting(data)

#generates embedding for the sentences
emb = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')
#emb = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embed2 = emb.encode(data, show_progress_bar = True)

Batches:   0%|          | 0/68 [00:00<?, ?it/s]

In [213]:
standardscalar = MinMaxScaler()
embed = standardscalar.fit_transform(embed)

In [245]:
latent_dim = 75 

class Autoencoders(Model):
    def __init__(self, latent_dim):
        super().__init__()
        self.encoder = Sequential(
            [
                Dense(175, activation = 'relu'),
                Dense(latent_dim, activation = 'relu')
            ]
        )
        self.decoder = Sequential(
            [
                Dense(75, activation = 'relu')
                Dense(175, activation = 'relu'),
                Dense(384, activation = 'sigmoid')
            ]
        )
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
autoencoder = Autoencoders(latent_dim)

In [246]:
autoencoder.compile(optimizer='adam', loss='mae', metrics = ['mae'])

In [247]:
autoencoder.fit(embed, embed, epochs=15, shuffle=True, validation_data = (embed2, embed2))

Epoch 1/15
6250/6250 [==============================] - 9s 1ms/step - loss: 0.0589 - mae: 0.0589 - val_loss: 0.5070 - val_mae: 0.5070
Epoch 2/15
6250/6250 [==============================] - 8s 1ms/step - loss: 0.0525 - mae: 0.0525 - val_loss: 0.5074 - val_mae: 0.5074
Epoch 3/15
6250/6250 [==============================] - 8s 1ms/step - loss: 0.0509 - mae: 0.0509 - val_loss: 0.5066 - val_mae: 0.5066
Epoch 4/15
6250/6250 [==============================] - 8s 1ms/step - loss: 0.0499 - mae: 0.0499 - val_loss: 0.5069 - val_mae: 0.5069
Epoch 5/15
6250/6250 [==============================] - 8s 1ms/step - loss: 0.0495 - mae: 0.0495 - val_loss: 0.5073 - val_mae: 0.5073
Epoch 6/15
6250/6250 [==============================] - 8s 1ms/step - loss: 0.0491 - mae: 0.0491 - val_loss: 0.5068 - val_mae: 0.5068
Epoch 7/15
6250/6250 [==============================] - 8s 1ms/step - loss: 0.0489 - mae: 0.0489 - val_loss: 0.5074 - val_mae: 0.5074
Epoch 8/15
6250/6250 [==============================] - 8s 1ms

In [233]:
autoencoder.summary()

Model: "autoencoders_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_17 (Sequential)  (None, 50)                76175     
                                                                 
 sequential_18 (Sequential)  (None, 384)               76509     
                                                                 
Total params: 152,684
Trainable params: 152,684
Non-trainable params: 0
_________________________________________________________________


In [234]:
encoder_layer = autoencoder.get_layer(autoencoder.layers[0].name)
reduced = encoder_layer.predict(embed2)
print(reduced.shape)

68/68 [==============================] - 0s 908us/step
(2145, 50)


In [235]:
def num_bins_Freedman_Diaconis(data):
    num_bins = []
    for i in range(data.shape[1]):
        iqr = stats.iqr(data[:,i])
        h = 2 * iqr * (len(data[:,i]) ** (-1/3))
        if(np.max(data[:,i]) == 0.0):
            print("Column: ", i)
        num_bins.append(ceil((np.max(data[:,i])-np.min(data[:,i]))/h))
    return num_bins

In [236]:
def histogram(data, bins, frequency = False):
    all_hist = []
    all_bins = []
    for i in range(data.shape[1]):
        hist, bin_edges = np.histogram(data[:,i], bins = bins[i], density = not frequency)
        all_hist.append(hist)
        all_bins.append(bin_edges)
    return all_hist, all_bins

In [237]:
def find_bin(bins, value):
    if value == bins[0]:
        return 0
    i = 1
    while value > bins[i]:
        i+=1
    return i-1

In [238]:
def probability(histograms, bins, data):
    probs = []
    for col in range(data.shape[1]):
        col_prob = []
        width = np.diff(np.array(bins[col]))[0]
        for row in range(data.shape[0]):
            idx = find_bin(bins[col], data[row][col])
            prob = histograms[col][idx] * width
            col_prob.append(prob)
        probs.append(col_prob)
    return np.array(probs).T

In [239]:
def sentence_surprisals(probs):
    surprisals = []
    for i in range(probs.shape[0]):
        surprisals.append(-1*sum(np.log2(probs[i])))
    return np.array(surprisals)

In [240]:
hists, bins = histogram(reduced, num_bins_Freedman_Diaconis(reduced))
probs = probability(hists, bins, reduced)
sen_surprisals = sentence_surprisals(probs)
rank_surprisal = np.flip(np.argsort(sen_surprisals))
sen_rankings = []
for i in rank_surprisal:
    sen_rankings.append(data[i])
with open("Autoencoder Sentence Rankings.txt", 'w') as f:
    for i in sen_rankings:
        f.write(i + "\n")
    f.close()

Column:  0


C:\Users\vaj1\AppData\Local\Temp\ipykernel_19048\3411464988.py:8: RuntimeWarning: invalid value encountered in double_scalars
  num_bins.append(ceil((np.max(data[:,i])-np.min(data[:,i]))/h))


ValueError: cannot convert float NaN to integer

In [241]:
print(np.where(reduced == np.nan))

(array([], dtype=int64), array([], dtype=int64))


In [242]:
print(ceil(0.0))

0


In [243]:
zeros = []
for i in range(reduced.shape[1]):
    if np.max(reduced[:,i]) == 0.0:
        zeros.append(i)

In [244]:
print(zeros)

[0, 2, 6, 10, 16, 19, 24, 25, 26, 31, 33, 34, 36, 37, 39, 42, 44, 46, 47, 48]


In [184]:
print(reduced)

[[0.         0.         0.78215295 ... 0.         0.         0.81141233]
 [0.         0.         0.73797715 ... 0.         0.         0.        ]
 [0.         0.         0.6183241  ... 0.         0.         1.8491237 ]
 ...
 [0.         0.         1.0148429  ... 0.         0.         1.8013648 ]
 [0.         0.         0.8144597  ... 0.         0.         1.1173072 ]
 [0.         0.         0.7686293  ... 0.         0.         0.        ]]


In [208]:
reduced_no_zeros = []
for i in range(reduced.shape[1]):
    if np.max(reduced[:,i]) != 0:
        reduced_no_zeros.append(reduced[:,i])
reduced_no_zeros = np.array(reduced_no_zeros).T

In [210]:
print(reduced_no_zeros.shape)

(2145, 32)
